# Cats Vs Dogs Image Binary Classification
## Demonstrates Data Augmentation and Transfer Learning

### [Partly derived from Tutorial by Laurence Moroney](https://www.tensorflow.org/tutorials/images/classification)

In [ ]:
# import tensorflow module. Check API version.
import tensorflow as tf
import numpy as np

print (tf.__version__)

# required for TF to run within docker using GPU (ignore otherwise)
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

## Load Image Data with Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define training data augmentation pipeline
train_datagen = ImageDataGenerator( rescale = 1.0/255.,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# load training data
TRAINING_DIR = '../data/deep_learning/cats-and-dogs/training/'
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=128,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

# define validation data augmentation pipeline
validation_datagen = ImageDataGenerator( rescale = 1.0/255.,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# load validation data
VALIDATION_DIR = '../data/deep_learning/cats-and-dogs/testing/'
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                         batch_size=128,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

# Define and Compile CNN Model

In [ ]:
# initialize the model
model = tf.keras.models.Sequential([
    # Conv Layer 1
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    # Conv Layer 2
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Conv Layer 3
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Conv Layer 4
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    
    # FC layer 1
    tf.keras.layers.Dense(512, activation='relu'), 

    # FC layer 2
    tf.keras.layers.Dense(512, activation='relu'), 

    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model
print("[INFO] compiling model...")
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"])

# print model summary
model.summary()

## Train Model

In [ ]:
# define callback function for training termination criteria
#accuracy_cutoff = 0.99
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if(logs.get('accuracy') > 0.90):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

# initialize training config
epochs = 100

# run training
print("[INFO] training...")
history = model.fit(train_generator, validation_data=validation_generator,
                    epochs=epochs, verbose=1, callbacks=[myCallback()])

## Evaluate Training Performance

### Expected Output

![accplot](images/accuracyCatsVsDogs.png) ![lossplot](images/lossCatsVsDogs.png)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# retrieve a list of list results on training and test data sets for each training epoch
acc      = history.history['accuracy']
val_acc  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs   = range(len(acc)) # get number of epochs

# plot training and validation accuracy per epoch
plt.plot(epochs, acc, label='train accuracy')
plt.plot(epochs, val_acc, label='val accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc="lower right")
plt.title('Training and validation accuracy')
plt.figure()

# plot training and validation loss per epoch
plt.plot(epochs, loss, label='train loss')
plt.plot(epochs, val_loss, label='val loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc="upper right")
plt.title('Training and validation loss')

# Transfer Learning Demo

## Load Pre-Trained Inception V3 Model 

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)

pre_trained_weights_file = '../data/deep_learning/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model.load_weights(pre_trained_weights_file)

pre_trained_model.trainable = False

pre_trained_model.summary()

## Extract part of Pre-Trained Model

In [ ]:
last_output = pre_trained_model.get_layer('mixed10').output

base_model = tf.keras.Model(pre_trained_model.input, last_output)

base_model.summary()

## Extent Base Model with Own Custom Layers

In [ ]:
# initialize the model
extended_model = tf.keras.Sequential([
    # base model (part of Inception V3)
    base_model,
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 

    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 

    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 

    # Only 1 output neuron.
    # It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model
print("[INFO] compiling model...")
extended_model.compile(optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["accuracy"])

# print model summary
extended_model.summary()

## Train Model

In [ ]:
# define callback function for training termination criteria
#accuracy_cutoff = 0.99
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if(logs.get('accuracy') > 0.90):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

# initialize training config
epochs = 30

# run training
print("[INFO] training...")
history = extended_model.fit(train_generator, validation_data=validation_generator,
                    epochs=epochs, verbose=1, callbacks=[myCallback()])